In [1]:
class Matrix(object):
    def __init__(self, size, data=[]):
        self.shape = size
        self._rows, self._cols = self.shape
        self._data = data
        
        
    def identity(self):
        if not self._rows == self._cols:
            raise ValueError('Identity matrix must be square')
        self._data = []
        for idx in range(self._rows):
            row = [0] * self._cols
            row[idx] = 1
            self._data.extend(row)
            
    def create_identity(self):
        i = Matrix((self._cols, self._cols))
        i.identity()
        return i
            
    def raw(self):
        return [self._data[x:x+self._cols] for x in range(0, len(self._data), self._cols)]
    
    def column(self, idx):
        return [row[idx] for row in self.raw()]
    
    def zero(self):
        row, col = self.shape
        self._data = [0] * (row * col)
        
    def one(self):
        row, col = self.shape
        self._data = [1] * (row * col)
        
    def dot(self, rhs):
        rows_rhs, cols_rhs = rhs.shape

        if not self._cols == rows_rhs:
            message = 'got ({0}, {1})'.format(self._cols, rows_rhs)
            raise ValueError('columns on left must equal rows on right, ' + message)

        p = Matrix((self._rows, cols_rhs))
        p.zero()

        for row in range(self._rows):
            for col in range(cols_rhs):
                p[row,col] = self._dot_vector(self[row], rhs.column(col))

        return p
    
    def transpose(self):
        row, col = self.shape
        self.shape = (col, row)
        self._rows, self._cols = self.shape
        
    def flatten(self):
        return self._data
            
    def _longest_str(self, str_seq):
        longest = 0
        for s in str_seq:
            longest = max(len(s), longest)
        return longest    
    
    def _pad(self, s, total):
        return s + ' ' * abs(len(s) - total)
    
    def _pad_row(self, row, total):
        return ' '.join([self._pad(value, total) for value in row])
    
    def _header(self, w, total):
        l = (total * w) + (w - 1)
        h = ' /' + ' ' * (l - 2) + '\\' + '\n'
        h += '/' + ' ' * l + '\\' + '\n'
        return h

    def _footer(self, w, total):
        l = (total * w) + (w - 1)
        f = '\n' + '\\' + ' ' * l + '/'
        f += '\n' + ' \\' + ' ' * (l - 2) + '/'
        return f
    
    def _dot_vector(self, l, r):
        return sum([t[0] * t[1] for t in list(zip(l, r))])
    
    def __str__(self):
        data = [str(item) for item in self._data]

        longest = self._longest_str(data)

        rows = [data[x:x+self._cols] for x in range(0, len(data), self._cols)]

        repr = self._header(self._cols, longest)
        repr += '\n'.join(['|' + self._pad_row(row, longest) + '|' for row in rows])
        repr += self._footer(self._cols, longest)

        return repr
    
    def __getitem__(self, key):
        if isinstance(key, int):
            lower = key * self._cols
            return self._data[lower:lower + self._cols]
        else:
            row, col = key
            return self._data[row * self._cols + col]
    
    def __setitem__(self, key, value):
        if isinstance(key, int):
            raise NotImplementedError()
        row, col = key
        self._data[row * self._cols + col] = value
        
    def __add__(self, rhs):
        if isinstance(rhs, self.__class__):
            sum_data = [x + y for (x, y) in list(zip(self.flatten(), rhs.flatten()))]
        else:
            sum_data = [x + rhs for x in self._data]
            
        return Matrix(self.shape, data=sum_data)
                       
    def __radd__(self, lhs):
        return self.__add__(lhs)
    
    def __sub__(self, rhs):
        if isinstance(rhs, self.__class__):
            diff_data = [x - y for (x, y) in list(zip(self.flatten(), rhs.flatten()))]
        else:
            diff_data = [x - rhs for x in self._data]
            
        return Matrix(self.shape, data=diff_data)
                       
    def __rsub__(self, lhs):
        return self.__sub__(lhs)
    
    def __mul__(self, rhs):
        if isinstance(rhs, self.__class__):
            prod_data = [x * y for (x, y) in list(zip(self.flatten(), rhs.flatten()))]
        else:
            prod_data = [x * rhs for x in self._data]
            
        return Matrix(self.shape, data=prod_data)
                       
    def __rmul__(self, lhs):
        return self.__mul__(lhs)
    
    def __neg__(self):
        return Matrix(self.shape, data=[-item for item in self._data])

In [2]:
M = Matrix((2,3), data=list(range(6)))
print(M)

 /   \
/     \
|0 1 2|
|3 4 5|
\     /
 \   /


In [3]:
N = Matrix((3, 4), data=list(range(12)))
print(N)

 /         \
/           \
|0  1  2  3 |
|4  5  6  7 |
|8  9  10 11|
\           /
 \         /


In [4]:
MN = M.dot(N)
print(MN)

 /         \
/           \
|20 23 26 29|
|56 68 80 92|
\           /
 \         /


In [5]:
MM = M.dot(M)
print(MM)

ValueError: columns on left must equal rows on right, got (3, 2)

In [6]:
MI = Matrix((M.shape[1], M.shape[1]))
MI.identity()
MMI = M.dot(MI)
print(MMI)

 /   \
/     \
|0 1 2|
|3 4 5|
\     /
 \   /


In [7]:
NI = N.create_identity()
NNI = N.dot(NI)
print(NNI)

 /         \
/           \
|0  1  2  3 |
|4  5  6  7 |
|8  9  10 11|
\           /
 \         /


In [8]:
P = Matrix((3, 2))
P.zero()
print(P)

 / \
/   \
|0 0|
|0 0|
|0 0|
\   /
 \ /


In [9]:
P.transpose()
print(P)

 /   \
/     \
|0 0 0|
|0 0 0|
\     /
 \   /


In [10]:
P.identity()

ValueError: Identity matrix must be square

In [11]:
N.raw()

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]

In [12]:
N[1]

[4, 5, 6, 7]

In [13]:
N[1, 2]

6

In [14]:
N[1, 2] = 10
print(N)

 /         \
/           \
|0  1  2  3 |
|4  5  10 7 |
|8  9  10 11|
\           /
 \         /


In [15]:
print(N)

 /         \
/           \
|0  1  2  3 |
|4  5  10 7 |
|8  9  10 11|
\           /
 \         /


In [16]:
print(N + 1)

 /         \
/           \
|1  2  3  4 |
|5  6  11 8 |
|9  10 11 12|
\           /
 \         /


In [17]:
V = Matrix((3, 4), data=list(range(1, 13)))
print(N + V)

 /         \
/           \
|1  3  5  7 |
|9  11 17 15|
|17 19 21 23|
\           /
 \         /


In [18]:
print(1 + N)

 /         \
/           \
|1  2  3  4 |
|5  6  11 8 |
|9  10 11 12|
\           /
 \         /


In [19]:
print(V + N)

 /         \
/           \
|1  3  5  7 |
|9  11 17 15|
|17 19 21 23|
\           /
 \         /


In [20]:
print(N - 1)

 /         \
/           \
|-1 0  1  2 |
|3  4  9  6 |
|7  8  9  10|
\           /
 \         /


In [21]:
print(1 - N)

 /         \
/           \
|-1 0  1  2 |
|3  4  9  6 |
|7  8  9  10|
\           /
 \         /


In [22]:
print(N * 2)

 /         \
/           \
|0  2  4  6 |
|8  10 20 14|
|16 18 20 22|
\           /
 \         /


In [23]:
print(2 * N)

 /         \
/           \
|0  2  4  6 |
|8  10 20 14|
|16 18 20 22|
\           /
 \         /


In [24]:
print(V - N)

 /         \
/           \
|1  1  1  1 |
|1  1  -3 1 |
|1  1  1  1 |
\           /
 \         /


In [25]:
print(V * N)

 /             \
/               \
|0   2   6   12 |
|20  30  70  56 |
|72  90  110 132|
\               /
 \             /


In [26]:
print(N - N)

 /     \
/       \
|0 0 0 0|
|0 0 0 0|
|0 0 0 0|
\       /
 \     /


In [32]:
negative_N = -N
negative_N.transpose()
print(negative_N)

 /         \
/           \
|0   -1  -2 |
|-3  -4  -5 |
|-10 -7  -8 |
|-9  -10 -11|
\           /
 \         /
